In [1]:
import numpy as np
from numpy import cos, sin

import matplotlib.pyplot as plt

import rebound

from scipy.linalg import lstsq
from scipy.optimize import leastsq
from scipy.optimize import minimize

import sys
sys.path.append('/Users/audreyburggraf/Desktop/CITA/CITA_PROJECT_CODE')

from functions_new_parameters import find_var_from_param, calculate_a_hat, normalized_residuals
from functions_new_parameters import signal_func, signal_func_2P, generate_parallax_signal
from functions_new_parameters import normalized_residuals, normalized_residuals_2P
from functions_new_parameters import find_chi_squared

In [2]:
P_orb_scaled = 0.3*np.array([12,29.4])
P_orb_scaled

array([3.6 , 8.82])

In [3]:
#true parameters
S_ = 3


inc = [0.022759093, 0.043388885]    # inclination in radians: (Jupiter, Saturn)



orbital_pars = (0,                     # 0 - Jupiter: e
                4.772463213,           # 1 - Jupiter: omega
                1.75503590062,         # 2 - Jupiter: Omega
                0,                     # 3 - Saturn: e
                5.8674129728,          # 4 - Saturn: omega
                1.98470185703,         # 5 - Saturn: Omega
    
                np.cos(np.mean(inc)))  # 6 - Jupiter and 2: cosi

jup_vars = find_var_from_param(orbital_pars[0], orbital_pars[1], orbital_pars[2], orbital_pars[6])
sat_vars = find_var_from_param(orbital_pars[3], orbital_pars[4], orbital_pars[5], orbital_pars[6])

# ----------------------------------------------- 

# setting truepars 
truepars = np.zeros((S_, 19))

truepars[:,0] = np.random.normal(0, 1.93925472e-10, S_)    # 0 - Delta alpha_0
truepars[:,1] = np.random.normal(0, 1.93925472e-10, S_)    # 1 - Delta delta 0 
truepars[:,2] = 2.3084641853871365e-07                     # 2 - mu_alpha
truepars[:,3] = 1.770935480191023e-07                      # 3 - mu_delta
truepars[:,4] = 9.699321049402031e-08                      # 4 - varpi
    
truepars[:,5] = jup_vars[0]                                # 5 - Jupiter: var 1
truepars[:,6] = jup_vars[1]                                # 6 - Jupiter: var 2
truepars[:,7] = jup_vars[2]                                # 7 - Jupiter: var 3
truepars[:,8] = jup_vars[3]                                # 8 - Jupiter: var 4
truepars[:,9] = 0.00095459,                                # 9 - Jupiter: m_planet
truepars[:,10] = P_orb_scaled[0]                           # 10 - Jupiter: P_orb 
truepars[:,11] = np.random.uniform(0, truepars[:,10], S_)  # 11 - Jupiter: t_peri
    
truepars[:,12] = sat_vars[0]                               # 12 - Saturn: var 1
truepars[:,13] = sat_vars[1]                               # 13 - Saturn: var 2 
truepars[:,14] = sat_vars[2]                               # 14 - Saturn: var 3 
truepars[:,15] = sat_vars[3]                               # 15 - Saturn: var 4 
truepars[:,16] = 0.0002857                                 # 16 - Saturn: m_planet
truepars[:,17] = P_orb_scaled[1]                           # 17 - Saturn: P_orb
truepars[:,18] = np.random.uniform(0, truepars[:,17], S_)  # 18 - Saturn: t_peri

### function

In [4]:
def big_func(pars, N, S, alpha0, delta0):
# ----------------------------- I N I T I A L - S T U F F --------------------------------------
    # empty arrays 
    noise = np.zeros((S,N))
    eta_true = np.zeros((S, N))
    eta_obs = np.zeros((S, N))
    
    # setting times and theta
    times = np.linspace(0, 5, N) 
    theta = np.linspace(0, 2*np.pi, N)
    
    # calculate a_hat from pars (parallax, mass, P_orb)
    a_hat_p1 = calculate_a_hat(pars[0,4], pars[0,9],  pars[0,10])
    a_hat_p2 = calculate_a_hat(pars[0,4], pars[0,16], pars[0,17])

    a_hat = (a_hat_p1 + a_hat_p2)/2

    # create array of errors 
    sigma_err = np.geomspace(0.1, 1, S)*a_hat

    # create noise
    for i in range(S):
        noise[i] = np.random.normal(0, sigma_err[i], N)

    # create true and observed data 
    for i in range(S):
        # true signal is signal with 2 planets  
        eta_true[i] = signal_func_2P(pars[i], alpha0, delta0, times, theta) 
        
        # observed signal is signal plus noise 
        eta_obs[i] = eta_true[i] + noise[i]
# ----------------------------------------------------------------------------------------------------------

    
#   ---------------------------------- N O - P L A N E T - F I T --------------------------------------
    # empty arrays
    M = np.zeros((N, 5))
    array = np.zeros((N, 5))
    np_chi_sq = np.zeros((S))
    np_best_fit_val = np.zeros((S, 5))
    
    # making Pi data 
    PI_ra, PI_dec = generate_parallax_signal(alpha0, delta0, 1, times)

    # filling matric M 
    for i in range(N):
        M[i,0] = cos(theta[i])
        M[i,1] = sin(theta[i])
        M[i,2] = cos(theta[i]) * times[i]
        M[i,3] = sin(theta[i]) * times[i]
        M[i,4] = cos(theta[i])*PI_ra[i] + sin(theta[i])*PI_dec[i]

    # finding the best fit values for the S samples 
    for i in range(S):
        np_best_fit_val[i], _, _, _ = lstsq(M, eta_obs[i]) 
        
    # find chi squared 
    for k in range(S):
        for i in range(N):
            x = np_best_fit_val[k] # x is equal to the kth row of np_best_fit_val
            for j in range(5):
                array[i,j] = M[i,j]*x[j]

        array_row_sums = np.sum(array, axis=1)   
        np_chi_sq[k] = np.sum((array_row_sums - eta_obs[k])**2/sigma_err[k]**2)
    # ----------------------------------------------------------------------------------------------------------
    
    
    # ----------------------------- O N E - P L A N E T - F I T ----------------------------------------------
    # creating arrays
    guess_P1 = np.zeros((S,12))
    guess_P2 = np.zeros((S,12))
    fitted_params_1P_P1 = np.zeros((S, 12))
    fitted_params_1P_P2 = np.zeros((S, 12))
    eta_best_P1 = np.zeros((S, N))
    eta_best_P2 = np.zeros((S, N))
    chi_sq_1P_P1 = np.zeros(S)
    chi_sq_1P_P2 = np.zeros(S)

    # bounds for other method
    bounds = ((-np.inf, np.inf),              # Delta alpha_0 
              (-np.inf,np.inf),               # Delta delta_0 
              (-np.inf,np.inf),               # mu_alpha 
              (-np.inf,np.inf),               # mu_delta 
              (0, np.inf),                    # varpi 
              (-1, 1),                        # var1  
              (-1, 1),                        # var 2
              (-np.sqrt(0.7), np.sqrt(0.7)),  # var3  
              (-np.sqrt(0.7), np.sqrt(0.7)),  # var 4 
              (0,1),                          # m_planet
              (-np.inf, np.inf),              # P_orb
              (-np.inf, np.inf))              # t_peri

    # dividing up the parameters 
    for s in range(S):
        for i in range(5):
            guess_P1[s,i] = truepars[s,i]
            guess_P2[s,i] = truepars[s,i]

    for s in range(S):
        for j in range(7):
            guess_P1[s,j+5] = truepars[s,j+5]   
            guess_P2[s,j+5] = truepars[s,j+12]
            
    for s in range(S):
        # getting best/fitted values 
        fitted_params_1P_P1[s], _, _, _, _ = leastsq(normalized_residuals, guess_P1[s], args=(alpha0, delta0, sigma_err[s], eta_obs[s], times, theta), full_output=1)
        fitted_params_1P_P2[s], _, _, _, _ = leastsq(normalized_residuals, guess_P2[s], args=(alpha0, delta0, sigma_err[s], eta_obs[s], times, theta), full_output=1)
        
        # if nan values then use other method 
        fn = lambda x: normalized_residuals(x,*args) @ normalized_residuals(x,*args)
        args = (alpha0, delta0, sigma_err[s], eta_obs[s], times, theta)

        if np.isnan(fitted_params_1P_P1[s]).any():
            result_min_P1 = minimize(fn, guess_P1[s], method='Nelder-Mead', bounds=bounds)
            fitted_params_1P_P1[s] = result_min_P1.x

        if np.isnan(fitted_params_1P_P2[s]).any():
            result_min_P2 = minimize(fn, guess_P2[s], method='Nelder-Mead', bounds=bounds)
            fitted_params_1P_P2[s] = result_min_P2.x
        
        # creating best signal from the best fit 
        eta_best_P1[s] = signal_func(fitted_params_1P_P1[s], alpha0, delta0, times, theta)
        eta_best_P2[s] = signal_func(fitted_params_1P_P2[s], alpha0, delta0, times, theta)
        
        # finding chi squared for with and without planet
        chi_sq_1P_P1[s] = find_chi_squared(eta_best_P1[s], eta_obs[s], sigma_err[s])
        chi_sq_1P_P2[s] = find_chi_squared(eta_best_P2[s], eta_obs[s], sigma_err[s])
    # -------------------------------------------------------------------------------------------------------------
    
    
    # ---------------------------------------- T W O - P L A N E T - F I T ----------------------------------------
    # creating arrays
    guess_2P = np.zeros((S,19))
    fitted_params_2P = np.zeros((S, 19))
    eta_best_2P = np.zeros((S, N))
    chi_sq_2P = np.zeros((S))

    # bounds for other method
    bounds_2P = ((-np.inf, np.inf),              # 0  - Delta alpha_0 
                 (-np.inf,np.inf),               # 1  - Delta delta_0 
                 (-np.inf,np.inf),               # 2  - mu_alpha 
                 (-np.inf,np.inf),               # 3  - mu_delta 
                 (0, np.inf),                    # 4  - varpi 
                 (-1, 1),                        # 5  - planet 1: var1  
                 (-1, 1),                        # 6  - planet 1: var 2
                 (-np.sqrt(0.7), np.sqrt(0.7)),  # 7  - planet 1: var3  
                 (-np.sqrt(0.7), np.sqrt(0.7)),  # 8  - planet 1: var 4 
                 (0,1),                          # 9  - planet 1: m_planet
                 (-np.inf, np.inf),              # 10 - planet 1: P_orb
                 (-np.inf, np.inf),              # 11 - planet 1: t_peri
                 (-1, 1),                        # 12 - planet 2: var1  
                 (-1, 1),                        # 13 - planet 2: var 2
                 (-np.sqrt(0.7), np.sqrt(0.7)),  # 14 - planet 2: var3  
                 (-np.sqrt(0.7), np.sqrt(0.7)),  # 15 - planet 2: var 4 
                 (0,1),                          # 16 - planet 2: m_planet
                 (-np.inf, np.inf),              # 17 - planet 2: P_orb
                 (-np.inf, np.inf))              # 18 - planet 2: t_peri

    # data where error varys 
    for s in range(S):
        # guess
        guess_2P[s] = truepars[s] 

        # arguments
        args = (alpha0, delta0, sigma_err[s], eta_obs[s], times, theta)
        
        # getting best/fitted values 
        fitted_params_2P[s], _, _, _, _ = leastsq(normalized_residuals_2P, guess_2P[s], args=(alpha0, delta0, sigma_err[s], eta_obs[s], times, theta), full_output=1)

        # if there are nan values then use other method 
        fn_2P = lambda x: normalized_residuals_2P(x,*args) @ normalized_residuals_2P(x,*args)

        if np.isnan(fitted_params_2P[s]).any():
            result_min_2P = minimize(fn,guess[s],method='Nelder-Mead', bounds=bounds_2P)
            fitted_params_2P[s] = result_min_2P.x

        # creating best signal from the best fit 
        eta_best_2P[s] = signal_func_2P(fitted_params_2P[s], alpha0, delta0, times, theta)
        
        # finding chi squared for with and without planet
        chi_sq_2P[s] = find_chi_squared(eta_best_2P[s], eta_obs[s], sigma_err[s])
    # -------------------------------------------------------------------------------------------------------------
        
        
    # ---------------------------- C A L C U L A T I N G - B I C - A N D - D E L T A - B I C---------------------
    Delta_BIC = np.zeros((5,S))
    
    BIC_np = np_chi_sq + 5 * np.log(N)
    BIC_1P_P1 = chi_sq_1P_P1 + 12*np.log(N)
    BIC_1P_P2 = chi_sq_1P_P2 + 12*np.log(N)
    BIC_2P = chi_sq_2P + 12*np.log(N)
    
    Delta_BIC[0] = BIC_1P_P1 - BIC_np     # 1 planet vs no planet - planet 1
    Delta_BIC[1] = BIC_1P_P2 - BIC_np     # 1 planet vs no planet - planet 2
    Delta_BIC[2] = BIC_2P    - BIC_np     # no planet vs 2 planets 
    Delta_BIC[3] = BIC_2P    - BIC_1P_P1  # 1 planet vs 2 planets - planet 1
    Delta_BIC[4] = BIC_2P    - BIC_1P_P2  # 1 planet vs 2 planets - planet 2
    # -------------------------------------------------------------------------------------------------------------

    return(a_hat, sigma_err, Delta_BIC)

In [5]:
big_func(truepars, N = 100, S = 3, alpha0 = 1, delta0 = 0.3)

ValueError: too many values to unpack (expected 3)

In [ ]:
sigma_err = np.zeros((S_))
Delta_BIC = np.zeros((5, S_))

a_hat, sigma_err, Delta_BIC = big_func(truepars, N = 100, S = 3, alpha0 = 1, delta0 = 0.3)